In [1]:
from pandas import DataFrame, Series
import pandas as pd
import numpy as np
from IPython.display import display

# 说明

关于pandas的笔记。 讲解面对常用的场景和需求时的解决方案。 不过并不力求全面。 

这份笔记的内容涉及Series的创建。 Series成员修改， 添加和删除。

# 创建
## 从已知的数据创建

定义一个Series最基本的两个参数是data和index，分别提供数据和对应的标签。 

可以分别传入两个等长的array like数据结构分别定义data和index，例如list, tuple, numpy.array, generator。

下面我们创建四个tuple，tuple中第一个成员之后用做index, 第二个成员用作data。 它们的类型分别是list, tuple, numpy.array, generator。 这种情况下data和index必须等长度。

In [2]:
inputs = [
    ([1,2], ['a','b']),
    ((1,2), ('a','b')),
    ((x for x in [1,2]), (x for x in ['a','b'])),
    (np.array([1,2]),np.array(['a','b']))    
]

遍历每一个tuple，创建Series。

In [3]:
for data, index in inputs:
    print('<<')    
    print('data的类型是',type(data), 'index的类型是',type(index))
    
    s = Series(data, index) #定义Series
    
    display(s)
    print('>>')

<<
data的类型是 <class 'list'> index的类型是 <class 'list'>


a    1
b    2
dtype: int64

>>
<<
data的类型是 <class 'tuple'> index的类型是 <class 'tuple'>


a    1
b    2
dtype: int64

>>
<<
data的类型是 <class 'generator'> index的类型是 <class 'generator'>


a    1
b    2
dtype: int64

>>
<<
data的类型是 <class 'numpy.ndarray'> index的类型是 <class 'numpy.ndarray'>


a    1
b    2
dtype: int64

>>


如果data传入单个数值,Series的长度会以传入的index为准。

In [4]:
s = Series(1,['a','b','c'])
s

a    1
b    1
c    1
dtype: int64

如果data是dict或一个Series的话，dict的keys或Series的index就可以作为index使用。这种情况下，可以省略index参数。 例如。

In [5]:
d = {'a':1,'b':2}

s = Series(d) #传入dict
s

a    1
b    2
dtype: int64

如果这时额外的指定index, 那么传入dict和Series定义的index会和传入index做匹配。 传入index中存在但是数据中找不到对应label数据会被用NaN填充。 传入index中不存在, 传入数据中存在的label会被舍弃。

In [6]:
d = {'a':1,'b':2}
s = Series(d, index = ['a','c'])
s # 'b'对应的数据被舍弃，'c'对应的数据用NaN填充

a    1.0
c    NaN
dtype: float64

总结:
    
依照传入data的类型， 传入的index会有不同的表现。

|data类型|index需要和data等长度|index用途|
|---|---|---|
|array like|是|定义|
|标量|否|定义|
|dict/Series|否|匹配|





### name

可以给Series传入一个name作为Series的名字。当Series组成DataFrame时,name会成为DataFrame的columns名称。(DataFrame后面会介绍)

In [7]:
d = {'a':1,'b':2}
s = Series(s,index = ['a','c'],name = 'a series')

print('Series:')
display(s)
print('Series name:')
display(s.name)

Series:


a    1.0
c    NaN
Name: a series, dtype: float64

Series name:


'a series'

### dtype

创建Series的时候可以传入多类型的数据,Series会尝试自动用一个适合所有成员的数据类型作为最终的数据类型。

例如下面的例子里,int类型和float类型放在一起时，所有的成员都被统一成了float。

In [8]:
s = Series([1,1.1])
display(s)

0    1.0
1    1.1
dtype: float64

下面的例子里，pandas则用了最万能的object类型作为Series的结构。这时每个成员保持了原有的数据类型。

In [9]:
s = Series([True,1,1.1,'1.23'])
display(s)

for x in s:
    print('数据:\t',x,'\t数据类型:',type(x))

0    True
1       1
2     1.1
3    1.23
dtype: object

数据:	 True 	数据类型: <class 'bool'>
数据:	 1 	数据类型: <class 'int'>
数据:	 1.1 	数据类型: <class 'float'>
数据:	 1.23 	数据类型: <class 'str'>


如果不满意pandas的自动处理策略，可以通过指定dtype的方式来指定最终的数据类型。

In [10]:
s = Series([True,1,1.1,'1.23'],dtype = float)
print('Series')
display(s)

for x in s:
    print('数据:\t',x,'\t数据类型:',type(x))

Series


0    1.00
1    1.00
2    1.10
3    1.23
dtype: float64

数据:	 1.0 	数据类型: <class 'numpy.float64'>
数据:	 1.0 	数据类型: <class 'numpy.float64'>
数据:	 1.1 	数据类型: <class 'numpy.float64'>
数据:	 1.23 	数据类型: <class 'numpy.float64'>


## 利用函数创建

可以利用一些函数创建常用的Sereis。 例如，连续的日期序列。

In [11]:
pd.date_range('1/1/2000', periods=8)

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07', '2000-01-08'],
              dtype='datetime64[ns]', freq='D')

# 选择成员

## 基于label进行选择

### 单个label

使用```at[]```， 传入单个label。

In [12]:
s = Series([1,2,3,4],index = ['a','b','c','d'])

result = s.at['a']

result

1

Series也可以像dict那样使用```get()```方法， 在找不到指定label成员的时候，可以返回一个默认值。

In [13]:
s.get('a',None)

1

### 多个label
#### loc[]

使用```loc[]```,传入list of label

In [14]:
result = s.loc[['a','c']]

result

a    1
c    3
dtype: int64

如果传入了并不存在的label,会用NaN填充

In [15]:
result = s.loc[['a','c','f']]

result

a    1.0
c    3.0
f    NaN
dtype: float64

但是如果所有的label都不存在,会报错。

In [16]:
try:
    result = s.loc[['g','f']]
except Exception as e:
    print(type(e),e)

<class 'KeyError'> "None of [['g', 'f']] are in the [index]"


```loc[]```也可以接受label slice

In [17]:
s = Series([1,2,3,4],index = ['a','c','b','d'])

s.loc['a':'c']

a    1
c    2
dtype: int64

注意'a':'c'并不是找出label是['a','b','c']的成员, 而是找到'a'后，按顺序向后搜索，直到找到'c'，并不假设label按照特定的顺序排过序。

下面这个例子，从'zda'对应的成员开始搜索，直至找到'dsdasd'对应的成员。 最后的:2表示返回一个成员后跳过1个成员, 不加的默认是:1,表示不跳过成员。

In [18]:
s = Series([1,2,3,4],index = ['zda','axa','dsdasd','a'])

s.loc['zda':'dsdasd':2]

zda       1
dsdasd    3
dtype: int64

#### reindex()

```reindex()```也可以返回多个label对应的成员， 如果找不到label对应的成员， 则用NaN填充。 和```loc```不同的是， 及时所有的label都找不到相应的成员，也不会报错。

In [19]:
s = Series([1,2,3,4],index = ['a','c','b','d'])

result = s.reindex(['g','f'])

result

g   NaN
f   NaN
dtype: float64

## 基于位置索引选择

位置索引从0开始递增。第一个成员索引是0,第二个成员索引是1，以此类推。

位置索引可以是负数，倒数第一个成员的索引是-1,倒数第二个成员的索引是-2，以此类推。


### 单个位置
使用```.iat[i]```。例如选择最后一个成员。

In [20]:
s = Series([1,2,3,4],index = ['a','c','b','d'])

s.iat[-1]

4

### 多个位置
使用```iloc[l]```的方式进行选择。```l```是位置索引组成的list。 例如选择第一个和最后一个成员。

In [21]:
s = Series([1,2,3,4],index = ['a','c','b','d'])

s.iloc[[0,-1]]

a    1
d    4
dtype: int64

不像```loc```,```iloc```中传入的位置索引如果超出Series的范围，不会自动扩展，而是抛出异常。

In [22]:
s = Series([1,2,3,4],index = ['a','c','b','d'])

try:
    s.iloc[9]
except Exception as e:
    print(type(e),e)

<class 'IndexError'> single positional indexer is out-of-bounds


```iloc[]```也支持传入slice。slice的语法是```m:n:s```。其中m是起始索引值（包含）, n是终止索引值（不包含）, s是步长(不填的话默认是1,不会跳过任何成员，2会跳过一个成员，以此类推)。 例如，要选出位置索引是0,1,2的成员，可以用

In [23]:
s = Series([1,2,3,4],index = ['a','c','b','d'])
s.iloc[0:3]

a    1
c    2
b    3
dtype: int64

选出索引是0,2成员，可以用

In [24]:
s = Series([1,2,3,4],index = ['a','c','b','d'])
s.iloc[0:4:2]

a    1
b    3
dtype: int64

如果slice的索引越界，会自动忽略越界的部分。不会用NaN填充，也不会抛出异常。

In [25]:
s = Series([1,2,3,4],index = ['a','c','b','d'])
s.iloc[0:100]

a    1
c    2
b    3
d    4
dtype: int64

slice也支持负数位置索引。 例如选出倒数第1个,跳过倒数第2个, 选出倒数第3个...，可以用。

In [26]:
s = Series([1,2,3,4],index = ['a','c','b','d'])
s.iloc[-1:-4:-2]

d    4
c    2
dtype: int64

总结:
    
```loc```,```iloc```,```reindex```对于越界的选择处理方式是不一样的。

|场景|行为|
|---|---|
|loc[label_list]|至少一个label存在，越界部分NaN填充;否则抛出异常|
|loc[slice]|报错|
|reindex(label_list)|越界部分NaN填充|
|iloc[int_list]|抛出异常|
|iloc[slice]|忽略越界部分|

```loc[]```和```iloc[]```中使用slice a:b的含义是不一样的，以a>0,b>0的情况为例。

||含义|
|---|---|
|loc[a:b]|找到a对应成员开始，直到找到c对应的成员|
|iloc[a:b]|找到位置索引在[a, b)之间的成员|

## 基于bool值

### loc[]

可以传入由True,False构成的list（长度需要和Series相同）进行选择。返回True对应位置成员组成的Series。例如下面这种方法，可以选出位置索引是0,2,3的成员。

In [27]:
result = s.loc[[True,False,True,True]]

result

a    1
b    3
d    4
dtype: int64

也可以往```loc[]```中传入bool Series。注意这时候会按照index进行匹配。下面的例子里，选出的成员是'c','b'对应的成员， 而不是前两个成员。

In [28]:
s = Series([1,2,3,4],index = ['a','c','b','d'])

condition = Series([True, True, False, False], index = ['c','b','a','d'])

s.loc[condition]

c    2
b    3
dtype: int64

用bool Series筛选的Series的时候。 bool Series的长度需要和待筛选的Series相同，否则会出错。

In [31]:
s = Series([1,2,3,4],index = ['a','c','b','d'])

condition = Series([True, True, False], index = ['c','b','a'])

try:
    s.loc[condition]
except Exception as e:
    print(type(e),e)

<class 'pandas.core.indexing.IndexingError'> Unalignable boolean Series key provided


bool Series一般对原有的Series进行条件判断得到的。

例如，要知道那些成员的数值大于1。

In [32]:
greater_then_one = (s>1)
greater_then_one

a    False
c     True
b     True
d     True
dtype: bool

判断哪些成员是偶数

In [33]:
is_even = s%2==0
is_even

a    False
c     True
b    False
d     True
dtype: bool

如果用```&```,```|```,```~```表示逻辑连词与，或，非。 从而将几个基本的条件组成复杂的条件。 例如,选出成员数值>1且是偶数的成员。

In [34]:
s.loc[greater_then_one&(is_even)]

c    2
d    4
dtype: int64

## 基于函数

```select()```方法可以接受一个函数。这个函数对index的label值进行判断，返回True,False值。 例如，找出label>'a'的成员

In [35]:
s.select(lambda x:x>'a')

c    2
b    3
d    4
dtype: int64

等价的方法是

In [36]:
s.loc[s.index>'a']

c    2
b    3
d    4
dtype: int64

```loc[]```中也支持使用函数，只要这个函数满足

* 接受一个Series
* 返回list of label或者和Series等长的bool list或者bool Series

使用函数的好处是可以减少一些中间临时变量， 更容易利用method chaining。 例如， 如果我们要分两次筛选出数值是偶数和index>='b'的成员。(这里是为了演示的目的故意将筛选分两次做，实际使用时可以将两个条件用&合并成一个)如果不利用函数，只能这么做。

In [37]:
s = Series([1,2,3,4],index = ['a','b','c','d'])

s1 = s.loc[s%2==0]

s1[s1.index>='b']

b    2
d    4
dtype: int64

利用函数的话，可以省掉临时变量s1

In [38]:
s = Series([1,2,3,4],index = ['a','b','c','d'])

s.loc[s%2==0].loc[lambda s:s.index>='b']

b    2
d    4
dtype: int64

```iloc[]```同理。

## 其它方法
Series提供了```[]```可以根据情况“智能的”进行成员选择。

In [39]:
s = Series([1,2,3,4],index = ['a','b','c','d'])

它可以像```loc[]```那样基于label进行选择。

In [40]:
s[['a','b','c','f']]

a    1.0
b    2.0
c    3.0
f    NaN
dtype: float64

如果index中找不到相应的label，而传入的是int list,那么它会像```iloc[]```那样基于位置参数进行选择。

In [41]:
s[[0,1]]

a    1
b    2
dtype: int64

它支持slice

In [42]:
s[0:3]

a    1
b    2
c    3
dtype: int64

支持单个label

In [43]:
s['a']

1

其他用法不一一展示了。```[]```能够根据情况自动的决定选择行为在使用的时候会带来一些便利，但是代价是看到这段代码的时候需要花费额外的精力去判断```[]```到底做了什么。在我们的例子中，由于Series的内容是可见的；可是生产代码中，别人看到的只是代表Series的变量，这时要准确判断```[]```的行为就很困难。

此外，pandas本身为了判断```[]```到底应该做什么，也需要花费额外的时间。

因此只推荐在数据探索阶段编写一些不会长期保存的代码时使用```[]```，不要在生产代码中使用。

同理，尽管```iloc[]```,```loc[]```也支持传入标量，起到和```iat[]```,```at```一样的功能。 但是建议```iloc[]```,```loc[]```只使用,传入标量时一律用```iat[]```和```at[]```，这样阅读代码时能立即明确意图。

## 随机采样

随机挑出一些样本，利用```sample()```函数。重要的参数有。

* n:样本数量

In [44]:
s = Series([1,2,3,4],index = ['a','b','c','d'])

s.sample(n=2)

c    3
d    4
dtype: int64

frac:样本数量的比例。

In [45]:
s.sample(frac = 0.5)

a    1
b    2
dtype: int64

随机数生成的seed， 给定的seed生成的随机结果在多次运行的时候会产生一样的结果，方便测试和debug。

In [46]:
for i in range(3):
    display(s.sample(n=2, random_state=1))

d    4
c    3
dtype: int64

d    4
c    3
dtype: int64

d    4
c    3
dtype: int64

In [47]:
for i in range(3):
    display(s.sample(n=2))

a    1
c    3
dtype: int64

b    2
a    1
dtype: int64

b    2
d    4
dtype: int64

# 修改

## 修改单个数值

用at或者iat选择单个成员并且用```=```赋值即可。例如将label为'c'的成员改成33， 第一个成员改成11，可以用。

In [48]:
s = Series([1,2,3,4],index = list('abcd'))

s.at['c'] = 33

s.iat[0] = 11

s

a    11
b     2
c    33
d     4
dtype: int64

### 修改多个成员

用loc或者iloc选择多个成员并且用```=```赋值即可， ```=```右侧需要是和```=```左侧等长度的array like object， 例如list。 例如将label为'a','b'的成员改成11,12, 第3,4个成员改成33和44。

In [157]:
s = Series([1,2,3,4],index = list('abcd'))

s.loc[['a','b']] = [11,22]

s.iloc[[2,3]] = [33,44]

s

a    11
b    22
c    33
d    44
dtype: int64

```=```右侧除了可以是和选出的成员数量等长度的list，也可以是一个标量，这样所有选中的成员都会被修改成同一个值。

In [158]:
s = Series([1,2,3,4],index = list('abcd'))

s.loc['a','b'] = 0

s

a    0
b    0
c    3
d    4
dtype: int64

```=```右侧甚至可以是一个Series或者dict。这种情况下,选中的成员会匹配```=```右侧的Series，用右侧Series同样label的成员数值替换自己的数值；如果在右侧找不到相应的label,则用NaN填充。

In [159]:
s = Series([1,2,3,4],index = list('abcd'))

s2 = Series([10,20,30,40],index = list('acde'))

s.loc['a','b'] = s2

s

a    10.0
b     NaN
c     3.0
d     4.0
dtype: float64

In [160]:
s = Series([1,2,3,4],index = list('abcd'))

s2 = {'a':10,'b':20,'c':30,'d':40}

s.loc['a','b'] = s2

s

a    10
b    20
c     3
d     4
dtype: int64

### where

利用Where可以使用不完整的bool Series作为筛选条件。

```where```的重要参数

* condition: 另一个存有Bool值的ndarray, Series, 或者是接受Series，返回前面两者的函数。
* other: 单一数值, 另一个Series， 或是返回前面两者的函数。

如果满足condition， 则会保留原来的Series的数值， 否则用other中的数值代替。

In [125]:
s = Series([1,2,3,4],index = list('abcd'))

s.where(np.array([True, True, False, False]), 10)

a     1
b     2
c    10
d    10
dtype: int64

下面使用Series作为condition的例子。 注意这里会采用index alignment, 在原Series存在label，在条件中不存在的label会从other中取数值替代。

In [127]:
s = Series([1,2,3,4],index = list('abcd'))

s.where(Series([True, True, False], index = list('abc')), 10)

a     1
b     2
c    10
d    10
dtype: int64

使用函数的例子。

In [130]:
s = Series([1,2,3,4],index = list('abcd'))

s.where(lambda s:s<=2, 10)

a     1
b     2
c    10
d    10
dtype: int64

other除了单一量，也可以是函数或者另一个Series。

按照下面的条件, 只有'a'对应的成员会被保留，其余的从other取。

In [140]:
s<2

a     True
b    False
c    False
d    False
dtype: bool

对于希望从other中取值， 但是other中找不到label的情况，会被用NaN填充。

In [141]:
other = Series([10,20],index = ['a','b'])

s.where(s<2, other)

a     1.0
b    20.0
c     NaN
d     NaN
dtype: float64

other是函数的例子。

In [144]:
s.where(s<2, lambda s:s*10)

a     1
b    20
c    30
d    40
dtype: int64

相比于用loc选择后赋值，where除了有自动label aligment的特性， 也可以用与将数据中的NaN替换成None。

In [145]:
s = Series([1,2,3,4,np.NaN])
s

0    1.0
1    2.0
2    3.0
3    4.0
4    NaN
dtype: float64

下面的方法是无效的。

In [152]:
s[s.isnull()] = None
s

0    1.0
1    2.0
2    3.0
3    4.0
4    NaN
dtype: float64

但是用where可以顺利进行。

In [153]:
s.where(s.notnull(), None)

0       1
1       2
2       3
3       4
4    None
dtype: object

## Mask
和```where```，用other的数值替换满足条件部分的数值。

In [155]:
s.mask(s.isnull(), None)

0       1
1       2
2       3
3       4
4    None
dtype: object

### 通过函数的方式修改
map

replace

where

apply

.str

.cat

### 新增成员

#### 增加单个成员

新增成员的语法和修改成员类似，但是只能使用```at```。

In [59]:
s = Series([1,2,3,4],index = list('abcd'))

s.at['e'] = 5

s

a    1
b    2
c    3
d    4
e    5
dtype: int64

但是不支持```iat[]```

In [60]:
s = Series([1,2,3,4],index = list('abcd'))

try:
    s.iat[4] = 5
except Exception as e:
    print(type(e),e)

<class 'IndexError'> index 4 is out of bounds for axis 0 with size 4


#### 增加多个成员
不能用loc或者iloc

In [61]:
s = Series([1,2,3,4],index = list('abcd'))

try:
    s.loc[['e','f']] = [5,6]
except Exception as e:
    print(type(e),e)

<class 'KeyError'> "['e' 'f'] not in index"


In [62]:
s = Series([1,2,3,4],index = list('abcd'))

try:
    s.iloc[[4,5]] = [5,6]
except Exception as e:
    print(type(e),e)

<class 'IndexError'> positional indexers are out-of-bounds


不过可以借助```append()```或者```pd.concat()```实现。

In [63]:
s_1 = Series([1,2,3,4],index = list('abcd'))
s_2 = Series([5,6],index = list('ef'))

s_1 = s.append(s_2)

s_1

a    1
b    2
c    3
d    4
e    5
f    6
dtype: int64

In [64]:
s_1 = Series([1,2,3,4],index = list('abcd'))
s_2 = Series([5,6],index = list('ef'))

s_1 = pd.concat([s_1,s_2])

s_1

a    1
b    2
c    3
d    4
e    5
f    6
dtype: int64

要注意append和concat等方法返回的Series并不是直接在原来的Series上做修改，而是返回一个新的Series。 只不过前面的例子里我们把返回的结果重新赋值给了```s_1```将```s_1```的指向变成了新的Series，所以起到了类似修改的效果。

在大部分情况下，这个差别并不会对我们的造成影响。但是如果代码的其它部分依然依赖```s_1```的数据的话，就要小心。

下面的例子中我们可以看到,```s_2```并没有发生改变。

In [65]:
s_1 = Series([1,2,3,4],index = list('abcd'))

s_2 = s_1

s_3 = Series([5,6],index = list('ef'))

s_1 = s_1.append(s_3)

s_2

a    1
b    2
c    3
d    4
dtype: int64

当然这个差别并不是Series特有的要注意的地方，在整个pandas的使用，甚至python编程的过程中，都应该关注一下object是被直接修改(modified in place)，还是返回了一个新的数据被改变过的object。

在pandas中，如果调用方法后返回了一个Series或者DataFrame， 大部分情况默认行为都是返回新的object。 部分函数支持```inplace```参数， 如果设成```True```就会直接修改原有的object, 返回None。 

## 删除
### 删除单个成员
#### del

可以用```del```运算符，直接从原有的Series中删除成员。

In [66]:
s = Series([1,2,3,4],index = list('abcd'))

del s['a']

s

b    2
c    3
d    4
dtype: int64

#### pop
```pop()```从原有的Sereis删除某个成员,并且返回被删除的成员。

In [67]:
s = Series([1,2,3,4],index = list('abcd'))

x = s.pop('a')

display(s)

display(x)

b    2
c    3
d    4
dtype: int64

1

#### drop
```drop()```返回删除一个或者多个成员后的新Series。

In [68]:
s1 = Series([1,2,3,4],index = list('abcd'))

s2 = s1.drop(['a','b'])

display(s1 is s2)

display(s2)

False

c    3
d    4
dtype: int64

drop也可以用来删除单个成员。

In [69]:
s1 = Series([1,2,3,4],index = list('abcd'))

s2 = s1.drop('a')

s2

b    2
c    3
d    4
dtype: int64

drop支持```inplace```参数， 设为True以后就会直接修改原来的Series, 返回None。

In [70]:
s1 = Series([1,2,3,4],index = list('abcd'))

result = s1.drop('a',inplace = True)
display(type(result))

s1

NoneType

b    2
c    3
d    4
dtype: int64